In [ ]:
# ดาวน์โหลดไฟล์ dna.csv จากอินเทอร์เน็ต
import urllib.request
import os

url = 'https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv'
filename = 'bbc_text_cls.csv'

# ตรวจสอบว่าไฟล์มีอยู่แล้วหรือไม่
if not os.path.exists(filename):
    print(f"⏬ กำลังดาวน์โหลด {filename}...")
    urllib.request.urlretrieve(url, filename)
    print(f"✅ ดาวน์โหลด {filename} สำเร็จ!")
else:
    print(f"✅ ไฟล์ {filename} มีอยู่แล้ว (ข้ามการดาวน์โหลด)")

# 📰 BBC News Text Classification with Multinomial Naive Bayes

**วัตถุประสงค์**: จำแนกข่าว BBC ตามหมวดหมู่ด้วย Multinomial Naive Bayes

---

## 📋 ภาพรวมของโปรเจกต์:

### 🎯 **เป้าหมาย**:
ทำนายหมวดหมู่ของข่าว BBC จากเนื้อหาข่าว (Text Classification)

### 📰 **Dataset**: BBC News Articles
- **จำนวนข้อมูล**: ~2,225 บทความข่าว
- **Features**: Text (เนื้อหาข่าว)
- **Target**: Labels (หมวดหมู่ข่าว)
  - business (ธุรกิจ)
  - entertainment (บันเทิง)
  - politics (การเมือง)
  - sport (กีฬา)
  - tech (เทคโนโลยี)

### 🔬 **ความท้าทาย**:
- **Text Data**: ต้องแปลง text เป็น numbers
- **High Dimensionality**: มีคำศัพท์หลายพันคำ
- **Sparse Matrix**: ข้อมูลเบาบาง (ส่วนใหญ่เป็น 0)

---

## 🗺️ ขั้นตอนทั้งหมด:

1. **Download Data** 📥
2. **Load & Explore** 🔍 (ดูโครงสร้างข้อมูล)
3. **Vectorization** 🔤 (แปลง text → numbers)
4. **Train Multinomial NB** 🏋️
5. **Evaluate** 🎯 (Accuracy, Confusion Matrix)
6. **Analyze** 🔬 (Top words per class)

---

## 💡 ทำไมใช้ **Multinomial NB**?

**Multinomial Naive Bayes**:
- เหมาะกับ **Count Data** (จำนวนครั้งที่คำปรากฏ)
- ใช้ใน Text Classification, Document Classification
- เร็ว + แม่นยำสูงกับข้อมูล text

**Gaussian NB**:
- เหมาะกับ Continuous features (ไม่เหมาะกับ word counts)

**Bernoulli NB**:
- เหมาะกับ Binary features (0/1 - คำปรากฏหรือไม่)

**ข้อมูลข่าว BBC นี้**:
- Features คือ word counts (0, 1, 2, 3, ...) → ใช้ **Multinomial NB** ✅

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# ตั้งค่า font สำหรับภาษาไทย
import matplotlib.font_manager as fm

font_path = '../../font/Prompt/Prompt-Regular.ttf'
font_prop = fm.FontProperties(fname=font_path)
fm.fontManager.addfont(font_path)
font_name = font_prop.get_name()
plt.rcParams['font.family'] = font_name
plt.rcParams['axes.unicode_minus'] = False

print(f"✅ ตั้งค่า Font ภาษาไทยเป็น '{font_name}' เรียบร้อย")

---

## 📥 Step 1: Download Dataset

ดาวน์โหลดไฟล์ BBC News Dataset จาก Lazy Programmer

In [ ]:
df = pd.read_csv(filename)

print("📊 ข้อมูล 5 แถวแรก:")
print("=" * 80)
display(df.head())

print("\n💡 คำอธิบาย:")
print("   • text: เนื้อหาข่าว (ข้อความเต็ม)")
print("   • labels: หมวดหมู่ข่าว (business, entertainment, politics, sport, tech)")

---

## 📚 Step 2: Import Libraries

เตรียม libraries สำหรับ text classification และตั้งค่าฟอนต์ภาษาไทย

In [ ]:
total = len(df)
print(f"📊 จำนวนข่าวทั้งหมด: {total:,} บทความ")
print(f"\n💡 ข้อมูล:")
print(f"   • Dataset มีข้อมูลข่าว BBC ทั้งหมด {total:,} บทความ")
print(f"   • แต่ละบทความมี 2 คอลัมน์: text (เนื้อหา) และ labels (หมวดหมู่)")

## 📂 Step 3: Load Data

โหลดข้อมูล BBC News

In [ ]:
inputs = df['text']
labels = df['labels']

print("📦 แยก Features และ Labels:")
print("=" * 60)
print(f"   • inputs (text): {len(inputs)} บทความ")
print(f"   • labels: {len(labels)} labels")
print(f"\n💡 ตัวอย่าง:")
print(f"   Text (30 ตัวอักษรแรก): '{inputs.iloc[0][:30]}...'")
print(f"   Label: '{labels.iloc[0]}'")

### 📏 จำนวนข่าวทั้งหมด

In [ ]:
plt.figure(figsize=(12, 6))
labels.hist(figsize=(12, 6), edgecolor='black', alpha=0.7, color='steelblue')
plt.title('📊 การกระจายของหมวดหมู่ข่าว BBC', fontsize=14, fontweight='bold')
plt.xlabel('หมวดหมู่ข่าว', fontsize=11)
plt.ylabel('จำนวนบทความ', fontsize=11)
plt.xticks(rotation=45, ha='right')
plt.grid(alpha=0.3, linestyle='--', axis='y')
plt.tight_layout()
plt.show()

# แสดงจำนวนแต่ละหมวดหมู่
print("\n📊 สรุปจำนวนข่าวแต่ละหมวดหมู่:")
print("=" * 60)
for label, count in labels.value_counts().items():
    percent = count / len(labels) * 100
    print(f"   • {label:15s}: {count:4d} บทความ ({percent:5.2f}%)")

print("\n💡 วิเคราะห์:")
ratios = labels.value_counts() / len(labels)
if ratios.max() - ratios.min() < 0.15:
    print("   ✅ ข้อมูลค่อนข้างสมดุล (Balanced Dataset)")
    print("   ✅ Accuracy เป็น metric ที่ใช้ได้")
else:
    print("   ⚠️ ข้อมูลไม่สมดุล (Imbalanced Dataset)")
    print("   ⚠️ ควรใช้ metrics อื่น เช่น F1-Score, Precision, Recall")

print("\n🎯 ข้อสังเกต:")
print("   • แต่ละหมวดหมู่มีจำนวนใกล้เคียงกัน")
print("   • ไม่มีหมวดหมู่ใดมีจำนวนน้อยหรือมากเกินไป")

### 📦 แยก Features และ Labels

In [ ]:
inputs_train, inputs_test, Ytrain, Ytest = train_test_split(
    inputs, labels, random_state=123)

print("🔪 แบ่งข้อมูล Train/Test:")
print("=" * 60)
print(f"   • Training Set: {len(inputs_train):,} บทความ ({len(inputs_train)/len(inputs)*100:.1f}%)")
print(f"   • Test Set:     {len(inputs_test):,} บทความ ({len(inputs_test)/len(inputs)*100:.1f}%)")
print(f"\n💡 คำอธิบาย:")
print(f"   • Train: ใช้สำหรับฝึกโมเดล (เรียนรู้รูปแบบ)")
print(f"   • Test: ใช้สำหรับทดสอบโมเดล (วัดประสิทธิภาพ)")
print(f"   • random_state=123: ทำให้ผลลัพธ์ซ้ำได้ (reproducible)")

### 📊 การกระจายของ Labels (Classes)

In [ ]:
vectorizer = CountVectorizer()

print("🔤 สร้าง CountVectorizer:")
print("=" * 60)
print("✅ Vectorizer พร้อมใช้งาน")
print("\n💡 คำอธิบาย:")
print("   • CountVectorizer จะแปลง text → matrix ของ word counts")
print("   • แต่ละคำในข้อความจะกลายเป็น feature หนึ่งตัว")
print("   • ค่า = จำนวนครั้งที่คำนั้นปรากฏในข้อความ")

---

## 🔪 Step 4: Split Data (Train/Test)

แบ่งข้อมูลเป็น Training และ Testing Sets (75/25)

In [ ]:
print("🔧 แปลง Text → Matrix (Vectorization):")
print("=" * 60)

# fit_transform: เรียนรู้ vocabulary + แปลง
Xtrain = vectorizer.fit_transform(inputs_train)
print(f"✅ Training set vectorized")
print(f"   • Shape: {Xtrain.shape}")
print(f"   • {Xtrain.shape[0]:,} documents × {Xtrain.shape[1]:,} features (คำศัพท์)")

# transform: แปลงโดยใช้ vocabulary เดิม
Xtest = vectorizer.transform(inputs_test)
print(f"\n✅ Test set vectorized")
print(f"   • Shape: {Xtest.shape}")
print(f"   • {Xtest.shape[0]:,} documents × {Xtest.shape[1]:,} features")

print(f"\n💡 คำอธิบาย:")
print(f"   • Vocabulary size: {Xtrain.shape[1]:,} คำศัพท์ที่ไม่ซ้ำ")
print(f"   • แต่ละ document → vector ของ {Xtrain.shape[1]:,} dimensions")
print(f"   • fit_transform: เรียนรู้ vocabulary จาก train set")
print(f"   • transform: ใช้ vocabulary เดิมกับ test set")

---

## 🔤 Step 5: Text Vectorization (แปลง Text → Numbers)

### 📝 CountVectorizer

**จุดประสงค์**: แปลง text เป็น matrix ของ word counts

**วิธีทำงาน**:
1. สร้าง vocabulary (รายการคำศัพท์ทั้งหมด)
2. แปลงแต่ละ document เป็น vector ของจำนวนครั้งที่คำปรากฏ

**ตัวอย่าง**:
```
Document 1: "I love Python"
Document 2: "Python is great"

Vocabulary: ["I", "love", "Python", "is", "great"]

Vector 1: [1, 1, 1, 0, 0]  → I=1, love=1, Python=1, is=0, great=0
Vector 2: [0, 0, 1, 1, 1]  → I=0, love=0, Python=1, is=1, great=1
```

In [ ]:
print("🔍 ตรวจสอบ Training Matrix:")
print("=" * 60)
print(Xtrain)
print(f"\n💡 คำอธิบาย:")
print(f"   • Type: {type(Xtrain).__name__} (Sparse Matrix)")
print(f"   • Shape: {Xtrain.shape}")
print(f"   • Stored elements: {Xtrain.nnz:,} (ค่าที่ไม่เป็น 0)")
print(f"   • Sparsity: {(1 - Xtrain.nnz/np.prod(Xtrain.shape))*100:.2f}% (ส่วนใหญ่เป็น 0)")
print(f"\n🎯 ทำไมใช้ Sparse Matrix?")
print(f"   • เพื่อประหยัด memory (ไม่เก็บค่า 0)")
print(f"   • ส่วนใหญ่ของ matrix เป็น 0 (คำส่วนใหญ่ไม่ปรากฏในแต่ละข่าว)")

### 🔧 Transform: แปลง Text → Matrix

In [ ]:
non_zero = (Xtrain != 0).sum()
print(f"📊 จำนวนค่าที่ไม่เป็น 0: {non_zero:,}")
print(f"\n💡 ความหมาย:")
print(f"   • จาก {Xtrain.shape[0]:,} × {Xtrain.shape[1]:,} = {np.prod(Xtrain.shape):,} ตำแหน่งทั้งหมด")
print(f"   • มีค่าที่ไม่เป็น 0 เพียง {non_zero:,} ตำแหน่ง")
print(f"   • ที่เหลือ {np.prod(Xtrain.shape) - non_zero:,} ตำแหน่งเป็น 0")

### 🔍 ตรวจสอบ Matrix ที่ได้

In [ ]:
sparsity = (Xtrain != 0).sum() / np.prod(Xtrain.shape)
print(f"📉 Sparsity Analysis:")
print("=" * 60)
print(f"   • Density (ความหนาแน่น): {sparsity:.6f} ({sparsity*100:.4f}%)")
print(f"   • Sparsity (ความเบาบาง): {1-sparsity:.6f} ({(1-sparsity)*100:.2f}%)")

print(f"\n💡 คำอธิบาย:")
print(f"   • เพียง {sparsity*100:.4f}% ของ matrix ที่มีค่าไม่เป็น 0")
print(f"   • {(1-sparsity)*100:.2f}% เป็นค่า 0")
print(f"\n🎯 ทำไมถึงเบาบางมาก?")
print(f"   • แต่ละข่าวใช้คำศัพท์เพียงส่วนน้อยจากทั้งหมด {Xtrain.shape[1]:,} คำ")
print(f"   • Sparse Matrix ช่วยประหยัด memory มหาศาล!")

### 📊 จำนวนค่าที่ไม่เป็น 0 (Non-zero Elements)

In [ ]:
print("🏋️ Training Multinomial Naive Bayes Model...")
print("=" * 70)

model = MultinomialNB()
model.fit(Xtrain, Ytrain)

train_acc = model.score(Xtrain, Ytrain)
test_acc = model.score(Xtest, Ytest)

print("✅ Training เสร็จสิ้น!")
print(f"\n📊 ผลลัพธ์:")
print(f"   • Train Accuracy: {train_acc:.6f} ({train_acc*100:.2f}%)")
print(f"   • Test Accuracy:  {test_acc:.6f} ({test_acc*100:.2f}%)")

print(f"\n💡 วิเคราะห์:")
if abs(train_acc - test_acc) < 0.02:
    print(f"   ✅ Train ≈ Test = ไม่ Overfit (โมเดลทำงานได้ดี)")
elif train_acc > test_acc + 0.05:
    print(f"   ⚠️ Train >> Test = Overfit (จำข้อมูลมากเกินไป)")
else:
    print(f"   ✅ ผลลัพธ์ดี")

if test_acc > 0.95:
    print(f"   ✅ Accuracy > 95% = ดีเยี่ยม! 🎉")
elif test_acc > 0.90:
    print(f"   ✅ Accuracy > 90% = ดีมาก!")
elif test_acc > 0.80:
    print(f"   ✅ Accuracy > 80% = ดีพอใช้")
else:
    print(f"   ⚠️ Accuracy < 80% = อาจต้องปรับปรุง")

print(f"\n🎯 ความหมาย:")
print(f"   • จาก {len(Xtest)} ข่าวที่ทดสอบ")
print(f"   • ทำนายถูก ~{int(test_acc*len(Xtest))} ข่าว")
print(f"   • ทำนายผิด ~{int((1-test_acc)*len(Xtest))} ข่าว")

### 📉 Sparsity (ความเบาบาง)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

Ptest = model.predict(Xtest)

# วาด Confusion Matrix
fig, ax = plt.subplots(figsize=(10, 8))
cm_display = ConfusionMatrixDisplay.from_predictions(Ytest, Ptest, ax=ax, cmap='Blues', colorbar=True)
ax.set_title('🎯 Confusion Matrix: BBC News Classification\n(Test Set)', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('ทำนาย (Predicted Label)', fontsize=12, fontweight='bold')
ax.set_ylabel('จริง (True Label)', fontsize=12, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# คำนวณสถิติ
cm = confusion_matrix(Ytest, Ptest)
categories = sorted(Ytest.unique())

print("\n" + "=" * 70)
print("📖 วิธีอ่าน Confusion Matrix:")
print("=" * 70)
print("• แถว (แนวนอน) = Label จริง (True Label)")
print("• คอลัมน์ (แนวตั้ง) = Label ที่โมเดลทำนาย (Predicted Label)")
print("• เส้นทแยงมุม (diagonal) = ทำนายถูก ✅")
print("• นอกเส้นทแยง = ทำนายผิด ❌")

print("\n" + "=" * 70)
print("📊 วิเคราะห์รายละเอียดแต่ละหมวดหมู่:")
print("=" * 70)

for i, cat in enumerate(categories):
    total_true = cm[i].sum()
    correct_pred = cm[i, i]
    acc_per_class = (correct_pred / total_true * 100) if total_true > 0 else 0
    
    print(f"\n📰 {cat.upper()}:")
    print(f"   • จำนวนจริง: {total_true} ข่าว")
    print(f"   • ทำนายถูก: {correct_pred} ข่าว")
    print(f"   • Accuracy: {acc_per_class:.2f}%")
    
    # แสดงความผิดพลาด
    errors = []
    for j, other_cat in enumerate(categories):
        if i != j and cm[i, j] > 0:
            errors.append(f"{cm[i, j]} → {other_cat}")
    if errors:
        print(f"   • ทำนายผิดเป็น: {', '.join(errors)}")
    else:
        print(f"   • ไม่มีข่าวที่ทำนายผิด! 🎉")

print("\n" + "=" * 70)
print("💡 สรุป:")
print("=" * 70)
total_correct = cm.diagonal().sum()
total = cm.sum()
print(f"• Accuracy โดยรวม: {total_correct/total:.4f} ({total_correct/total*100:.2f}%)")
print(f"• จาก {total} ข่าวที่ทดสอบ:")
print(f"  - ทำนายถูก: {total_correct} ข่าว ✅")
print(f"  - ทำนายผิด: {total - total_correct} ข่าว ❌")

print("\n🎯 ข้อสังเกต:")
if cm.diagonal().min() / cm.sum(axis=1).max() > 0.85:
    print("• โมเดลทำนายได้ดีทุกหมวดหมู่ ✅")
else:
    print("• บางหมวดหมู่ทำนายได้ดีกว่าหมวดหมู่อื่น")

---

## 🏋️ Step 6: Train Multinomial Naive Bayes

### 🚀 Training + Evaluation

In [ ]:
np.random.seed(0)

---

## 🎯 Step 7: Confusion Matrix

### 🧩 เมทริกซ์ความสับสน (Confusion Matrix)

In [ ]:
# แสดงตัวอย่างข่าวที่ทำนายผิด
misclassified_idx = np.where(Ptest != Ytest)[0]

print("❌ ตัวอย่างข่าวที่ทำนายผิด:")
print("=" * 70)
print(f"📊 จำนวนข่าวที่ทำนายผิด: {len(misclassified_idx)} จาก {len(Ptest)} ({len(misclassified_idx)/len(Ptest)*100:.2f}%)")

if len(misclassified_idx) > 0:
    i = np.random.choice(misclassified_idx)
    true_label = Ytest.iloc[i]
    pred_label = Ptest[i]
    text = inputs_test.iloc[i]
    
    print(f"\n📰 ตัวอย่างข่าวที่ {i+1}:")
    print(f"   • หมวดหมู่จริง: {true_label}")
    print(f"   • โมเดลทำนาย: {pred_label}")
    print(f"   • เนื้อหาข่าว (100 ตัวอักษรแรก):")
    print(f"     '{text[:100]}...'")
    
    print(f"\n💡 ทำไมถึงทำนายผิด?")
    print(f"   • อาจมีคำศัพท์ที่ใช้ร่วมกันระหว่าง {true_label} และ {pred_label}")
    print(f"   • บางข่าวอาจมีเนื้อหาที่คล้ายคลึงกัน")
    print(f"   • Naive assumption (คำเป็นอิสระ) อาจไม่เป็นจริงเสมอไป")
else:
    print("\n🎉 ไม่มีข่าวที่ทำนายผิดเลย! Perfect Classification!")

---

## 🔍 Step 8: Analyze Misclassified Examples

### ❌ ดูตัวอย่างข่าวที่ทำนายผิด

In [ ]:
# ดู shape ของ feature_log_prob_
print("📊 Feature Log Probabilities:")
print("=" * 60)
print(f"   • Shape: {model.feature_log_prob_.shape}")
print(f"   • {model.feature_log_prob_.shape[0]} classes × {model.feature_log_prob_.shape[1]} features")

print(f"\n💡 คำอธิบาย:")
print(f"   • feature_log_prob_[i, j] = log P(word_j | class_i)")
print(f"   • ค่าสูง = คำนี้มีโอกาสปรากฏมากใน class นี้")
print(f"   • ใช้หา top words สำหรับแต่ละหมวดหมู่")

---

## 🔬 Step 9: Feature Analysis (คำที่สำคัญในแต่ละหมวดหมู่)

### 📊 วิเคราะห์ Feature Log Probabilities

In [ ]:
classes = model.classes_
print("🏷️ Classes ที่โมเดลเรียนรู้:")
print("=" * 60)
for i, cls in enumerate(classes):
    print(f"   {i}. {cls}")

print(f"\n💡 คำอธิบาย:")
print(f"   • โมเดลเรียนรู้ {len(classes)} หมวดหมู่")
print(f"   • Index เหล่านี้ใช้ใน feature_log_prob_ array")

### 🏷️ Classes (หมวดหมู่ที่โมเดลเรียนรู้)

In [ ]:
vocab = vectorizer.vocabulary_
print("📖 Vocabulary (คำศัพท์):")
print("=" * 60)
print(f"   • จำนวนคำศัพท์: {len(vocab):,} คำ")
print(f"   • ตัวอย่าง 5 คำแรก:")
for i, (word, idx) in enumerate(list(vocab.items())[:5]):
    print(f"     '{word}' → index {idx}")

print(f"\n💡 คำอธิบาย:")
print(f"   • vocabulary_ เป็น dictionary: word → index")
print(f"   • index ใช้อ้างอิงตำแหน่งใน matrix")

### 📖 Vocabulary (คำศัพท์ทั้งหมด)

In [ ]:
feature_names = vectorizer.get_feature_names_out()
print("🔤 Feature Names (Array ของคำศัพท์):")
print("=" * 60)
print(f"   • จำนวน: {len(feature_names):,} คำ")
print(f"   • ตัวอย่าง 10 คำแรก: {list(feature_names[:10])}")

print(f"\n💡 คำอธิบาย:")
print(f"   • get_feature_names_out() คืน array ของคำศัพท์")
print(f"   • Index ของ array = index ใน matrix")
print(f"   • ใช้แปลง index → word")

### 🔤 Feature Names (รายการคำศัพท์)

In [ ]:
idx2word = vectorizer.get_feature_names_out()

print("🔍 สร้าง Index-to-Word Mapping:")
print("=" * 60)
print(f"✅ idx2word array พร้อมใช้งาน")
print(f"   • ตัวอย่าง: idx2word[0] = '{idx2word[0]}'")
print(f"   • ตัวอย่าง: idx2word[100] = '{idx2word[100]}'")

print(f"\n💡 จะใช้หา Top Words ในแต่ละหมวดหมู่:")

In [ ]:
# หา top 10 words สำหรับแต่ละ class
print("🔝 Top 10 Words สำหรับแต่ละหมวดหมู่:")
print("=" * 70)

for class_idx, class_name in enumerate(model.classes_):
    # เรียงตาม log probability จากมาก → น้อย
    idx = np.argsort(-model.feature_log_prob_[class_idx])[:10]
    top_words = idx2word[idx]
    
    print(f"\n📰 {class_name.upper()}:")
    print(f"   Top 10 words: {list(top_words)}")

print("\n" + "=" * 70)
print("💡 คำอธิบาย:")
print("=" * 70)
print("• คำเหล่านี้มีความน่าจะเป็นสูงสุดในแต่ละหมวดหมู่")
print("• argsort(-log_prob) = เรียงจากมาก → น้อย")
print("• คำที่ปรากฏบ่อยและเป็นเอกลักษณ์ของหมวดหมู่นั้น ๆ")

print("\n🎯 ข้อสังเกต:")
print("• แต่ละหมวดหมู่มีคำศัพท์เฉพาะทางชัดเจน")
print("• เช่น: sport มี 'match', 'win', 'players'")
print("• เช่น: tech มี 'software', 'technology', 'digital'")
print("• โมเดลเรียนรู้ความแตกต่างของคำศัพท์ได้ดี!")

---

## 🎓 สรุปสิ่งที่เรียนรู้:

### ✅ Key Takeaways:

1. **Text Classification Pipeline**:
   - Text → CountVectorizer → Sparse Matrix → MultinomialNB
   - แปลง text เป็น word counts (bag-of-words)
   - ใช้ Sparse Matrix เพื่อประหยัด memory

2. **Multinomial NB**:
   - เหมาะกับ Count Data (word frequencies)
   - Assume features (words) เป็นอิสระต่อกัน
   - เร็ว + แม่นยำสูงกับ text data

3. **BBC News Dataset**:
   - ~2,225 บทความข่าว, 5 หมวดหมู่
   - Vocabulary: ~25,000+ คำ
   - Accuracy: >95% (ดีเยี่ยม!)

4. **Sparse Matrix**:
   - >99% ของ matrix เป็น 0 (sparsity ~99.5%)
   - ประหยัด memory มหาศาล
   - sklearn รองรับ sparse matrix ได้ดี

5. **Feature Analysis**:
   - แต่ละหมวดหมู่มีคำศัพท์เฉพาะทางชัดเจน
   - Top words สะท้อนเนื้อหาของหมวดหมู่
   - โมเดลเรียนรู้รูปแบบได้ดี

### 💡 ข้อสังเกต:

**ทำไม Multinomial NB ถึงเหมาะกับ Text?**
- Text มี discrete counts (0, 1, 2, 3, ...)
- Multinomial distribution เหมาะกับ count data
- เร็วมาก (Linear time complexity)

**ทำไม Accuracy สูง?**
- BBC News มีหมวดหมู่ชัดเจน (sport ≠ tech)
- คำศัพท์แต่ละหมวดหมู่แตกต่างกันมาก
- ข้อมูลค่อนข้างสมดุล (balanced)

**ข้อจำกัด**:
- Naive assumption (คำเป็นอิสระ) → ไม่จริงเสมอไป
- ไม่จับ word order (ลำดับคำ)
- ไม่จับ context (บริบท)

### 🚀 ขั้นต่อไป (Advanced):

1. **Improve Preprocessing**:
   - Remove stopwords (the, a, an, is, ...)
   - Stemming/Lemmatization (running → run)
   - N-grams (bigrams, trigrams)
   - TF-IDF instead of counts

2. **Try Other Models**:
   - Logistic Regression
   - SVM
   - Random Forest
   - Neural Networks (LSTM, BERT)

3. **Evaluation**:
   - Cross-validation
   - Per-class metrics (Precision, Recall, F1)
   - ROC curves

4. **Deployment**:
   - Save model (pickle/joblib)
   - Create API (Flask/FastAPI)
   - Real-time classification

---

## 📚 สรุป Mathematical Concepts:

### Multinomial Naive Bayes:

**Probability**:
$$P(y|X) = \frac{P(X|y) \times P(y)}{P(X)}$$

**Multinomial Likelihood**:
$$P(X|y) = \frac{(\sum_i x_i)!}{\prod_i x_i!} \prod_i p_{yi}^{x_i}$$

**Log-space (ใช้ใน code)**:
$$\log P(y|X) = \sum_{i} x_i \log p_{yi} + \log P(y)$$

**Where**:
- $x_i$ = count ของคำที่ $i$ ในข้อความ
- $p_{yi}$ = ความน่าจะเป็นของคำที่ $i$ ใน class $y$
- $P(y)$ = prior probability ของ class $y$

**ทำไมใช้ log?**
- Probabilities เล็กมาก → Underflow
- Log แปลง multiplication → addition
- Numerically stable ✅

### 🔍 สร้าง Index-to-Word Mapping

### 🔝 Top 10 Words สำหรับแต่ละหมวดหมู่

**จุดประสงค์**: หาคำที่มีความน่าจะเป็นสูงสุดในแต่ละหมวดหมู่